In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('train.csv')
df.head().append(df.tail())
df.shape

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.7500,NaN,Q


(891, 12)

## 2.数据重构
数据可能分布在多个文件或数据库中，或不易于分析的格式进行排列。本章节将对数据进行规整，总共分为六个版块：分层索引、联合与合并、重塑和透视，数据聚合以及常见应用（拆分-应用-联合）。

### 2.1 分层索引
分层索引是Pandas重要的基础特性，即允许在一个轴向上拥有多个索引层级，提供了一种在更低维度的形式中处理更高维度数据的方式。

In [4]:
### 案例说明
data = pd.DataFrame(np.random.rand(9),index=[['a','a','b','b','b','c','c','d','d'],[1,2,3,1,2,3,4,1,2]],
                   columns=['A'])
data  #两层index

#分层索引的意义-部分索引
'''外层索引-c部分'''
data.loc['c']
'''区间索引-从c到d'''
data['c':'d']
'''部分截取索引-a和c'''
data.loc[['a','c']]

A
a 1  0.606800
  2  0.559276
b 3  0.286061
  1  0.890549
  2  0.631398
c 3  0.210002
  4  0.052272
d 1  0.354587
  2  0.954045

'外层索引-c部分'

,A
3,0.210002
4,0.052272


'区间索引-从c到d'

A
c 3  0.210002
  4  0.052272
d 1  0.354587
  2  0.954045

'部分截取索引-a和c'

A
a 1  0.606800
  2  0.559276
c 3  0.210002
  4  0.052272

In [5]:
### 分组索引在重塑数据和数字透视表等分组操作-unstack&stack
'''unstack()意为不要'堆叠'，意将数据的列索引拆为行索引（默认），
stack()意为'堆叠'，意将行索引均堆为列索引（默认）'''
data.unstack()          #index拆分到columns，对应没有的项用NaN补充
data.unstack().stack()

"unstack()意为不要'堆叠'，意将数据的列索引拆为行索引（默认），\nstack()意为'堆叠'，意将行索引均堆为列索引（默认）"

A                              
          1         2         3         4
a  0.606800  0.559276       NaN       NaN
b  0.890549  0.631398  0.286061       NaN
c       NaN       NaN  0.210002  0.052272
d  0.354587  0.954045       NaN       NaN

A
a 1  0.606800
  2  0.559276
b 1  0.890549
  2  0.631398
  3  0.286061
c 3  0.210002
  4  0.052272
d 1  0.354587
  2  0.954045

In [6]:
###多层索引可以重命名
frame = pd.DataFrame(np.arange(12).reshape(4,3),
                    index = [['b','a','b','a'],[1,2,1,2]],
                    columns = [['O','O','H'],['Y','U','I']])
frame

frame.index.names =['in1','in2']    #多重索引重命名
frame.columns.names =['col1','col2']#重命名多重columns
frame

O       H
     Y   U   I
b 1  0   1   2
a 2  3   4   5
b 1  6   7   8
a 2  9  10  11

col1     O       H
col2     Y   U   I
in1 in2           
b   1    0   1   2
a   2    3   4   5
b   1    6   7   8
a   2    9  10  11

In [7]:
##直接用rename重命名尝试
frame.rename(columns = {'Y':'1','O':'2','col1':'kan'},inplace = True)
frame
'''单层索引的重命名可行,多重索引直接用index/columns.names赋值'''

col1     2       H
col2     1   U   I
in1 in2           
b   1    0   1   2
a   2    3   4   5
b   1    6   7   8
a   2    9  10  11

'单层索引的重命名可行,多重索引直接用index/columns.names赋值'

In [37]:
###换个角度看数据表
df.stack()
df.unstack()

0    PassengerId                          1
     Survived                             0
     Pclass                               3
     Name           Braund, Mr. Owen Harris
     Sex                               male
                             ...           
890  SibSp                                0
     Parch                                0
     Ticket                          370376
     Fare                              7.75
     Embarked                             Q
Length: 9826, dtype: object

PassengerId  0      1
             1      2
             2      3
             3      4
             4      5
                   ..
Embarked     886    S
             887    S
             888    S
             889    C
             890    Q
Length: 10692, dtype: object

### 2.1.1 重排序和层级排序
一些时候重新排列轴上的层及顺序，或按照特定层级的值对数据进行排序。

对于多重索引而言，有几个必备的参数需要理解应用：
- `levels`:每个等级上轴标签的唯一值。
- `labels`:以整数来表示每个level上标签的位置。
- `sortorder`:按照指定level的标签名称的字典顺序进行排序（可选参数）。
- `names`:index level的名称。
- `copy`:布尔值默认为False,是否拷贝元数据产生新的对象。

重排序和层级排序的函数：
- `swaplevel`接受两个层级序号或层级名称，返回一个进行了层级变更的新对象，数据不变。
- `sort_index`只能是单一层级对数据排序

In [8]:
### 参数案例说明
data = pd.Series(np.random.randn(10),
              index=[['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'd', 'd'],
                     [1, 2, 3, 1, 2, 3, 1, 2, 2, 3]])

data
data.index

a  1   -0.864354
   2    0.752598
   3    0.830085
b  1   -0.558941
   2    1.319583
   3   -3.062891
c  1    0.466355
   2    1.101642
d  2   -0.195975
   3    1.700688
dtype: float64

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 2),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

#### 上述详解
1. level中的唯一标签值集合分别为['a','b','c','d']和[1,2,3].
2. `level = 0`的标签为'a,b,c,d'对应的索引为'0,1,2,3'; `level = 1`的标签为'1,2,3'对应的索引为'0,1,2'.
3. 外层`level=0`的`label`值对应为[0,0,0,1,1,1,2,2,3,3]表示外层的标签出现的位置及次数。
4. 内层`level=1`的`label`值对应为[0,1,2,0,1,2,0,1,1,2]表示内层的标签出现的位置。

In [9]:
### swaplevel案例说明
frame
frame.swaplevel(axis=0) #index内外层级进行了交换，括号内也可以改为(in1,in2)或(0,1)
'''交换索引后可以外部索引2，如果直接索引2因其是内层的索引所以不成立'''
frame.swaplevel(axis=0).loc[2] 

col1     2       H
col2     1   U   I
in1 in2           
b   1    0   1   2
a   2    3   4   5
b   1    6   7   8
a   2    9  10  11

col1     2       H
col2     1   U   I
in2 in1           
1   b    0   1   2
2   a    3   4   5
1   b    6   7   8
2   a    9  10  11

'交换索引后可以外部索引2，如果直接索引2因其是内层的索引所以不成立'

col1  2       H
col2  1   U   I
in1            
a     3   4   5
a     9  10  11

In [10]:
### sort_index应用说明
frame
frame.sort_index(level=0) #同过level指定第0个index进行排序，或者level='in1'
frame.sort_index(level=1) #第1个index进行排序

col1     2       H
col2     1   U   I
in1 in2           
b   1    0   1   2
a   2    3   4   5
b   1    6   7   8
a   2    9  10  11

col1     2       H
col2     1   U   I
in1 in2           
a   2    3   4   5
    2    9  10  11
b   1    0   1   2
    1    6   7   8

col1     2       H
col2     1   U   I
in1 in2           
b   1    0   1   2
    1    6   7   8
a   2    3   4   5
    2    9  10  11

In [11]:
##案例应用
frame.swaplevel(0,1).sort_index(level=0)
frame.swaplevel(0,1).sort_index(level=1)

col1     2       H
col2     1   U   I
in2 in1           
1   b    0   1   2
    b    6   7   8
2   a    3   4   5
    a    9  10  11

col1     2       H
col2     1   U   I
in2 in1           
2   a    3   4   5
    a    9  10  11
1   b    0   1   2
    b    6   7   8

In [12]:
### 按层级进行汇总统计
frame.sum(level='in2')

col1   2       H
col2   1   U   I
in2             
1      6   8  10
2     12  14  16

### 2.1.2 使用DataFrame的列进行索引
这个也比较常用有些意思，在数据集中发现列有独特的属性，可以以这列作为索引。
- `set_index(['list1,list2'],drop=True)`:以list1,list2为索引，默认drop为True即移除这两列。
- `reset_index`:`set_index`的反操作，分层索引的索引层级会被移动到列中。

In [13]:
df_name_age = df.set_index(['Name','Age'],drop=True)
df_name_age.head()

,,PassengerId,Survived,Pclass,Sex,SibSp,Parch,Ticket,Fare,Cabin,Embarked
Name,Age,,,,,,,,,,
"Braund, Mr. Owen Harris",22.0,1,0,3,male,1,0,A/5 21171,7.2500,NaN,S
"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",38.0,2,1,1,female,1,0,PC 17599,71.2833,C85,C
"Heikkinen, Miss. Laina",26.0,3,1,3,female,0,0,STON/O2. 3101282,7.9250,NaN,S
"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,4,1,1,female,1,0,113803,53.1000,C123,S
"Allen, Mr. William Henry",35.0,5,0,3,male,0,0,373450,8.0500,NaN,S


In [14]:
### reset_index是set_index的反操作，分层索引的索引层次会被移到列中：
df_new = df_name_age.reset_index()
df_new.head()

,Name,Age,PassengerId,Survived,Pclass,Sex,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,"Braund, Mr. Owen Harris",22.0,1,0,3,male,1,0,A/5 21171,7.2500,NaN,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,2,1,1,female,1,0,PC 17599,71.2833,C85,C
2,"Heikkinen, Miss. Laina",26.0,3,1,3,female,0,0,STON/O2. 3101282,7.9250,NaN,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,4,1,1,female,1,0,113803,53.1000,C123,S
4,"Allen, Mr. William Henry",35.0,5,0,3,male,0,0,373450,8.0500,NaN,S


### 2.2 联合与合并数据集
- `pd.merge()`:可以根据一个或多个键将行进行连接，对于SQL或其他关系型数据库的连接实现。
- `join()`:可以应用到字符串连接，也可以应用到DataFrame中索引相同或相似的合并。
- `pd.concat()`:使对象在轴向上进行黏合或“堆叠”。
- `combine_first`:实例方法允许将重叠的数据接在一起，使用一个对象中的值填充另一个对象中的缺失值。


### 2.2.1 数据库风格的DataFrame连接
合并或连接操作通过一个或多个**键**连接行来联合数据集（关系型数据库的核心内容，例如SQL）。pandas的`merge`函数主要用于将各种join操作算法运用到数据上。

In [31]:
### 将几张表进行简单合并
text_l_u = pd.read_csv("train-left-up.csv")
text_l_d = pd.read_csv("train-left-down.csv")
text_r_u = pd.read_csv("train-right-up.csv")
text_r_d = pd.read_csv("train-right-down.csv")

In [32]:
## 查看下数据集
text_l_u.head()
text_l_u.shape
text_l_d.head()
text_l_d.shape
text_r_u.head()    #对于原整体数据而言，这块明显不是按份切割（后面函数应用有坑需避）
text_r_u.shape
text_r_d.head()
text_r_d.shape

,PassengerId,Survived,Pclass,Name
0,1,0,3,"Braund, Mr. Owen Harris"
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th..."
2,3,1,3,"Heikkinen, Miss. Laina"
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)"
4,5,0,3,"Allen, Mr. William Henry"


(439, 4)

,PassengerId,Survived,Pclass,Name
0,440,0,2,"Kvillner, Mr. Johan Henrik Johannesson"
1,441,1,2,"Hart, Mrs. Benjamin (Esther Ada Bloomfield)"
2,442,0,3,"Hampe, Mr. Leon"
3,443,0,3,"Petterson, Mr. Johan Emil"
4,444,1,2,"Reynaldo, Ms. Encarnacion"


(452, 4)

,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,male,22.0,1.0,0.0,A/5 21171,7.2500,NaN,S
1,female,38.0,1.0,0.0,PC 17599,71.2833,C85,C
2,female,26.0,0.0,0.0,STON/O2. 3101282,7.9250,NaN,S
3,female,35.0,1.0,0.0,113803,53.1000,C123,S
4,male,35.0,0.0,0.0,373450,8.0500,NaN,S


(891, 8)

,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,male,31.0,0,0,C.A. 18723,10.500,NaN,S
1,female,45.0,1,1,F.C.C. 13529,26.250,NaN,S
2,male,20.0,0,0,345769,9.500,NaN,S
3,male,25.0,1,0,347076,7.775,NaN,S
4,female,28.0,0,0,230434,13.000,NaN,S


(452, 8)

#### 连接与合并函数介绍:pd.merge()
`pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)`
- `left`: 拼接的左侧DataFrame对象
- `right`: 拼接的右侧DataFrame对象
- `how`:可取inner、left、righ和outer，分别表示键的交集、左表的键、右表的键、两张表都有的键进行联合。
- `on`: 要加入的列或索引级别名称。 必须在左侧和右侧DataFrame对象中找到。 如果未传递且left_index和right_index为False，则DataFrame中的列的交集将被推断为连接键。
- `left_on`:左侧DataFrame中的列或索引级别用作键。 可以是列名，索引级名称，也可以是长度等于DataFrame长度的数组。
- `right_on`: 右侧DataFrame中的列或索引级别用作键。 可以是列名，索引级名称，也可以是长度等于DataFrame长度的数组。
- `left_index`: 如果为True，则使用左侧DataFrame中的索引（行标签）作为其连接键。 对于具有MultiIndex（分层）的DataFrame，级别数必须与右侧DataFrame中的连接键数相匹配。
- `right_index`: 与left_index功能相似。
- **how: One of ‘left’, ‘right’, ‘outer’, ‘inner’.** 默认inner。inner是取交集，outer取并集。
- `sort`: 按字典顺序通过连接键对结果DataFrame进行排序。 默认为True，设置为False将在很多情况下显着提高性能。
- `suffixes`: 用于重叠列的字符串后缀元组。 默认为（‘x’，’ y’）。
- `copy`: 始终从传递的DataFrame对象复制数据（默认为True），即使不需要重建索引也是如此。
- `indicator`:将一列添加到名为_merge的输出DataFrame，其中包含有关每行源的信息。 _merge是分类类型，并且对于其合并键仅出现在“左”DataFrame中的观察值，取得值为left_only，对于其合并键仅出现在“右”DataFrame中的观察值为right_only，并且如果在两者中都找到观察点的合并键，则为left_only。

In [17]:
### 案例说明-直接连接
left = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                       'A': ['A0', 'A1', 'A2', 'A3'],
                       'B': ['B0', 'B1', 'B2', 'B3']})
right = pd.DataFrame({'key': ['K1', 'K1', 'K2', 'K3'],
                        'C': ['C0', 'C1', 'C2', 'C3'],
                        'D': ['D0', 'D1', 'D2', 'D3']})
result = pd.merge(left, right,on=['key'])
'''对于未指定在哪一列上进行连接的，merge会自动将重叠列名作为连接的键，
并且连接的键是共有的key,默认how=inner'''
result

'对于未指定在哪一列上进行连接的，merge会自动将重叠列名作为连接的键，\n并且连接的键是共有的key,默认how=inner'

,key,A,B,C,D
0,K1,A1,B1,C0,D0
1,K1,A1,B1,C1,D1
2,K2,A2,B2,C2,D2
3,K3,A3,B3,C3,D3


In [18]:
### how参数的应用
result_l = pd.merge(left,right,how='left')
'''以左键为主，扩展联合'''
result_l
'''以右键为主，扩展联合'''
result_r = pd.merge(left,right,how='right',on=['key'])
result_r
'''所有的键的并集进行联合'''
result_o = pd.merge(left,right,how='outer')
result_o
'''相同的键的并集被联合，suffiex默认重叠列的名字,共2*3=6项'''
left_1 = pd.DataFrame({'A' : [1,2], 'B' : [2, 2]})
right_1 = pd.DataFrame({'A' : [4,5,6], 'B': [2,2,2]})
result_1 = pd.merge(left_1, right_1, on='B', how='outer')
result_1

'以左键为主，扩展联合'

,key,A,B,C,D
0,K0,A0,B0,NaN,NaN
1,K1,A1,B1,C0,D0
2,K1,A1,B1,C1,D1
3,K2,A2,B2,C2,D2
4,K3,A3,B3,C3,D3


'以右键为主，扩展联合'

,key,A,B,C,D
0,K1,A1,B1,C0,D0
1,K1,A1,B1,C1,D1
2,K2,A2,B2,C2,D2
3,K3,A3,B3,C3,D3


'所有的键的并集进行联合'

,key,A,B,C,D
0,K0,A0,B0,NaN,NaN
1,K1,A1,B1,C0,D0
2,K1,A1,B1,C1,D1
3,K2,A2,B2,C2,D2
4,K3,A3,B3,C3,D3


'相同的键的并集被联合，suffiex默认重叠列的名字,共2*3=6项'

,A_x,B,A_y
0,1,2,4
1,1,2,5
2,1,2,6
3,2,2,4
4,2,2,5
5,2,2,6


In [19]:
### indicator的应用-信息的来源
df1 = pd.DataFrame({'col1': [0, 1], 'col_left':['a', 'b']})
df2 = pd.DataFrame({'col1': [1, 2, 2],'col_right':[2, 2, 2]})
pd.merge(df1, df2, on='col1', how='outer', indicator=True)

,col1,col_left,col_right,_merge
0,0,a,NaN,left_only
1,1,b,2.0,both
2,2,NaN,2.0,right_only
3,2,NaN,2.0,right_only


In [20]:
### index为链接键应用
left = pd.DataFrame({'A': ['A0', 'A1', 'A2'],
                     'B': ['B0', 'B1', 'B2']},
                    index=['K0', 'K1', 'K2'])
   
right = pd.DataFrame({'C': ['C0', 'C2', 'C3'],
                     'D': ['D0', 'D2', 'D3']},
                     index=['K0', 'K2', 'K3'])

'''只有K0、K2有对应的值'''
pd.merge(left,right,how= 'inner',left_index=True,right_index=True)

'''需要同时设置left_index= True 和 right_index= True，或者left_index设置的同时，right_on指定某个Key。
总的来说就是需要指定left、right链接的键，可以同时是key、index或者混合使用。'''

other = pd.DataFrame({'C': ['C0', 'C1'],
                      'D': ['D0', 'D1'],
                     'key': ['K0', 'K1']})
result_2 = pd.merge(other,right,left_on='key',right_index=True,how='outer',sort=False)
'''sort对链接的键值进行排序'''
result_2

'只有K0、K2有对应的值'

,A,B,C,D
K0,A0,B0,C0,D0
K2,A2,B2,C2,D2


'需要同时设置left_index= True 和 right_index= True，或者left_index设置的同时，right_on指定某个Key。\n总的来说就是需要指定left、right链接的键，可以同时是key、index或者混合使用。'

'sort对链接的键值进行排序'

,C_x,D_x,key,C_y,D_y
0.0,C0,D0,K0,C0,D0
1.0,C1,D1,K1,NaN,NaN
NaN,NaN,NaN,K2,C2,D2
NaN,NaN,NaN,K3,C3,D3


In [21]:
### 纯使用merge方法合并上述四张数据表
result_up = pd.merge(text_l_u,text_r_u,left_index=True, right_index=True)
result_down = pd.merge(text_l_d,text_r_d,left_index=True, right_index=True)
list = df.columns.tolist()              #调取数据集的所有columns转化成列表
result = pd.merge(result_up,result_down,on=list,how='outer')
'''将所有的columns作为key，然后进行Outer链接，即表格的上下连接，此处用append()也可，append()用于在列表末尾添加新的对象，即result=result_up.append(result_down)'''
result.head()
result.shape

'将所有的columns作为key，然后进行Outer链接，即表格的上下连接，此处用append()也可，append()用于在列表末尾添加新的对象，即result=result_up.append(result_down)'

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1.0,0.0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1.0,0.0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0.0,0.0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1.0,0.0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0.0,0.0,373450,8.0500,NaN,S


(891, 12)

### 2.2.2 根据索引合并
函数：join()

`join()`函数：是python自带用处非常广的函数，既可以用于字符串的连接，又可以DataFrame的合并，以下为其常用方式。
- `join`的字符串连接：`l_new = l_1.join(l2)`表示将l_1应用到l_2中连接成新的l_new.
- `df.join(other, on=None, how=’left’, lsuffix=”, rsuffix=”, sort=False)`：可以合并多个索引相同或相似但没有重叠列的DataFrame对象。
 - `other`:DataFrame，或者带有名字的Series，或者DataFrame的list,如果传递的是Series，那么其name属性应当是一个集合，并且该集合将会作为结果DataFrame的列名。
 - `lsuffix`:左DataFrame中重复列的后缀。
 - `rsuffix`:右DataFrame中重复列的后缀。
 - 其余参数在上述的`merge()`函数有介绍。

In [22]:
### join的字符串案例
l_1 =""                #什么都米有哈哈
l_2 = ['1','0','0','0','.','0','0']
l_new = l_1.join(l_2)  #将l_1应用到l_2中用作连接
l_new

'1000.00'

In [23]:
### join的DataFrame案例
left = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                              'B': ['B0', 'B1', 'B2', 'B3']},
                              index= ['K0', 'K1', 'K0', 'K1'])

right = pd.DataFrame({'A': ['C0', 'C1','C2'],
                               'D': ['D0', 'D1','D2']},
                              index=['K0', 'K1','k2'])
right2 = pd.DataFrame({'v': [7, 8, 9]}, index=['K1', 'K1', 'K2'])
left
right
right2

'''默认inner连接'''
left.join([right,right2]) 
'''outer一下'''
left.join([right,right2],how='outer')

,A,B
K0,A0,B0
K1,A1,B1
K0,A2,B2
K1,A3,B3


,A,D
K0,C0,D0
K1,C1,D1
k2,C2,D2


,v
K1,7
K1,8
K2,9


'默认inner连接'

,A_x,B,A_y,D,v
K0,A0,B0,C0,D0,NaN
K0,A2,B2,C0,D0,NaN
K1,A1,B1,C1,D1,7.0
K1,A1,B1,C1,D1,8.0
K1,A3,B3,C1,D1,7.0
K1,A3,B3,C1,D1,8.0


'outer一下'

,A_x,B,A_y,D,v
K0,A0,B0,C0,D0,NaN
K0,A2,B2,C0,D0,NaN
K1,A1,B1,C1,D1,7.0
K1,A1,B1,C1,D1,8.0
K1,A3,B3,C1,D1,7.0
K1,A3,B3,C1,D1,8.0
K2,NaN,NaN,NaN,NaN,9.0
k2,NaN,NaN,C2,D2,NaN


In [24]:
###用join函数和append函数合并上述四张表格
result_up = text_l_u.join(text_r_u)
result_down = text_l_d.join(text_r_d)
'''因为join函数是根据索引合并，所以在对于行的合并而言是可行的，但是后续的竖着合并需要append函数辅助，或者merge()函数'''
result_1 = result_up.append(result_down)
result_1.head()
result_1.shape

'因为join函数是根据索引合并，所以在对于行的合并而言是可行的，但是后续的竖着合并需要append函数辅助，或者merge()函数'

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1.0,0.0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1.0,0.0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0.0,0.0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1.0,0.0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0.0,0.0,373450,8.0500,NaN,S


(891, 12)

### 2.2.3 沿轴向连接：pd.concat()
`pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)`
          
| 参数       | 描述         | 应用    |
|:-----------:| :-------------:|:-------------:|
| `objs` | 要连接的对象，必选参数  | 首填，[]内，两个起 |
| `axis` | 连接的轴向  | 0为行方向，1为纵方向 |
| `join` | 指定连接的方式（内外连接）  | `inner`为内连接-交集，`outer`为外连接 |
| `join_axes` | 指定其他n-1轴的特定索引，可替代内外连接逻辑  | 单个或多个[]内部填写想知道的轴 |
| `keys` | 连接对象的数据集，沿着连接轴形成分层索引  | 例如多个数据表连接，常与names连用 |
| `names` | 若传入了Keys或levels，用于层级名称  | `names=['',''] `|
| `verify_integrity` | 检查连接的轴是否包含重复项  | boolean,默认False|
| `leels` | 键值传递时，用于指定多层索引的层级  | 暂无，用的少，目测可以结合level |
| `ignore_index` | 不沿着连接轴保留索引，产生新的索引  | `ignore_index`=True|

In [25]:
### 用concat函数合并上述四张表格
up = pd.concat([text_l_u,text_r_u],axis=1)
down = pd.concat([text_l_d,text_r_d],axis=1)
'''利用先左后右合并axis=0即默认，再左右拼接也是可行的。'''
result_2 = pd.concat([up,down])
result_2.head()
result_2.shape

'利用先左后右合并axis=0即默认，再左右拼接也是可行的。'

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1.0,0.0,3.0,"Braund, Mr. Owen Harris",male,22.0,1.0,0.0,A/5 21171,7.2500,NaN,S
1,2.0,1.0,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1.0,0.0,PC 17599,71.2833,C85,C
2,3.0,1.0,3.0,"Heikkinen, Miss. Laina",female,26.0,0.0,0.0,STON/O2. 3101282,7.9250,NaN,S
3,4.0,1.0,1.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1.0,0.0,113803,53.1000,C123,S
4,5.0,0.0,3.0,"Allen, Mr. William Henry",male,35.0,0.0,0.0,373450,8.0500,NaN,S


(1343, 12)

#### 注意：concat函数应用结果的不同
此处的合并后的数据集大小和原数据集不一样，与`merge`和`join`函数应用的结果都不同，为什么呢？

答：因为`merge`和`join`在合并的时候都默认`inner`形式合并，在左右连接的时候就已经把“重复默许去掉了”，此时得到的结果再进行上下拼接则是原数据表的完整拼接。而`concat`函数在第一次左右连接的时候，默认`outer`形式，所以把带有缺失值的也合并到数据集中。

### 2.2.4 联合重叠数据
`combine_first`:数据联合场景，非合并非连接，但是可“互补”，即有两个数据集，其索引全部或者部分重叠时。

In [41]:
### 案例说明
a = pd.DataFrame({'a':[1,np.nan,5,np.nan],'b':[np.nan,2,3,4],'c':range(2,10,2)})
b = pd.DataFrame({'a':[5,3,np.nan,1,2,9],'b':[np.nan,2,8,3,4,7]})
a
b
'''用b填充a'''
b.combine_first(a)

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,4
2,5.0,3.0,6
3,NaN,4.0,8


,a,b
0,5.0,NaN
1,3.0,2.0
2,NaN,8.0
3,1.0,3.0
4,2.0,4.0
5,9.0,7.0


'用b填充a'

,a,b,c
0,5.0,NaN,2.0
1,3.0,2.0,4.0
2,5.0,8.0,6.0
3,1.0,3.0,8.0
4,2.0,4.0,NaN
5,9.0,7.0,NaN


### 3. 数据聚合与运算
对数据进行分类，并在每一组上应用一个聚合函数或转换函数，是数据分析工作流中的一个重要部分。在载入、合并、准备数据后，需要计算分组统计或者数据透视表用于可视化。

pandas提供灵活的`groupby`接口，允许以一种自然的方式对数据集进行切片、切块和总结。

### Groupby()函数
结合《利用Python进行数据分析》及网络资料的整合（具体参考文献记录），对Groupby()函数有了一定的了解，感觉是对pandas数据应用最重要的环节并且最有意思，对于数据分析是必备的大餐！下面对数据聚合与分析的应用函数进行了一个简单地介绍。

首先结合资料介绍下groupby函数的常见用法：

| 函数	| 适用场景	| 备注| 
|:-----------:| :-------------:|:-------------:|
| `df.groupby('key')` |	一列聚合 |	分组键为列名（可以是字符串、数字或其他Python对象） |
| `df.groupby(['key1','key2'])` |	多列聚合	 |分组键为列名，引入列表`list[]` |
| `df['data1'].groupby(df['key1']).mean()`	 |按某一列进行一重聚合求均值 |	分组键为Series |
|` A=df['订单编号'].groupby([ df['运营商'], df['分类'], df['百度圣卡'] ]).count()`	 |按某一列进行多重聚合计数 |	分组键为Series，引入列表`list[]`
| `df['data1'].groupby([states,years]).mean()` |	分组键与原df无关，而是另外指定的任何长度适当的数组	 |分组键是数组，state和year均为数组 |
| `for indexes,data in df.groupby('data1'):...print(indexes)...print(data)` |	对分组进行迭代，产生一个由分组变量名和数据块组成的元组	 |输出的结果按照外层的index分开对data1列分组输出数据块 |
| `for (in1,in2),data in df.groupby(['data1','data2']):...print(in1,in2)...print(data)` |	对多分组进行迭代，产生一个由分组变量名和数据块组成的元组	 |输出的结果按照多层的index分开对data1和data2列分组输出数据块 |

#### Groupby()基本原理：
- 根据DataFrame本身的某一列或多列内容进行分组聚合。
- 若按某一列聚合，则新DataFrame将根据某一列的内容分为不同的维度进行拆解，同时将同一维度的再进行聚合。
- 若按某多列聚合，则新DataFrame将是多列之间维度的笛卡尔积，即：新DataFrame具有一个层次化索引（由唯一的键对组成），例如：“key1”列，有a和b两个维度，而“key2”有one和two两个维度，则按“key1”列和“key2”聚合之后，新DataFrame将有四个group。
- 注：`groupby()`默认是在`axis=0`上进行分组的，通过设置`axis=1`，也可以在其他任何轴上进行分组。

In [53]:
###举个栗子
df = pd.DataFrame({'key1':['a','a','b','b','a'],'key2':['one','two','one','two','one'],
                  'data1':np.random.randn(5),'data2':np.random.randn(5)})
df
df.dtypes
df.groupby(df.dtypes,axis=1).sum()
df.groupby(['key1','key2'])['data1'].sum()

,key1,key2,data1,data2
0,a,one,1.671553,-0.383206
1,a,two,0.206522,0.011985
2,b,one,0.236159,-0.668623
3,b,two,-0.122353,-0.305833
4,a,one,0.409347,0.689769


key1      object
key2      object
data1    float64
data2    float64
dtype: object

,float64,object
0,1.288347,aone
1,0.218507,atwo
2,-0.432463,bone
3,-0.428186,btwo
4,1.099116,aone


key1  key2
a     one     2.080901
      two     0.206522
b     one     0.236159
      two    -0.122353
Name: data1, dtype: float64

#### groupby()匹配函数-agg,apply,transfrom
- `agg，transform、apply`三个方法的输入对象，都是分组后的DataFrame/Series.区别在于其输出类型不一样。
- `agg`输出的是缩减后的标量（或者标量列表）。
- `transform`输出的是原输入的DataFrame大小的，但是数据元素经过了转换的DataFrame.
- `apply`很灵活，既可以是缩减后的标量，也可以是pandas对象。

In [58]:
### 案例说明
rng=np.random.RandomState(0)
df=pd.DataFrame({'key':['A','B','C','A','C','B'],'data1':range(6),'data2':rng.randint(0,10,6)})
df

#创建一个分组的迭代器
'''看看分组迭代器'''
grouped=df.groupby('key') 
for (key,group) in grouped:
    print(key)
    print(group)

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,C,4,7
5,B,5,9


'看看分组迭代器'

A
  key  data1  data2
0   A      0      5
3   A      3      3
B
  key  data1  data2
1   B      1      0
5   B      5      9
C
  key  data1  data2
2   C      2      3
4   C      4      7


In [59]:
###三个函数应用
'''agg'''
grouped.agg(sum)
'''transform'''
grouped.transform(sum)
'''apply'''
grouped.apply(sum)

'agg'

,data1,data2
key,,
A,3,8
B,6,9
C,6,10


'transform'

,data1,data2
0,3,8
1,6,9
2,6,10
3,3,8
4,6,10
5,6,9


'apply'

,data1,data2
key,,
A,3,8
B,6,9
C,6,10


#### 上述小结
- apply和agg的功能是重叠的，只是apply多了一列。
- 因为agg处理的是分组后的所有列的和，而apply处理的是分组后，传入apply的函数的DataFrame的所有列。
- transform和apply及agg的区别就很清楚了，它是把所有列求和的值，然后再返回原数据结构，所以返回的数据结构大小是不变的，这一点有其很特殊的地位。

In [43]:
### 案例带入
text = pd.read_csv('result.csv')
text.head()
text.shape

,Unnamed: 0,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1.0,0.0,A/5 21171,7.2500,NaN,S
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1.0,0.0,PC 17599,71.2833,C85,C
2,2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0.0,0.0,STON/O2. 3101282,7.9250,NaN,S
3,3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1.0,0.0,113803,53.1000,C123,S
4,4,5,0,3,"Allen, Mr. William Henry",male,35.0,0.0,0.0,373450,8.0500,NaN,S


(891, 13)

In [60]:
###计算泰坦尼克号男性与女性的平均票价
price_mean = text['Fare'].groupby(text['Sex']).mean()
price_mean

Sex
female    44.479818
male      25.523893
Name: Fare, dtype: float64

In [62]:
###统计泰坦尼克号中男女的存活人数
survived_sex = text['Survived'].groupby(text['Sex']).sum()
survived_sex

Sex
female    233
male      109
Name: Survived, dtype: int64

In [65]:
###计算客舱不同等级的存活人数
survived_Pclass = text.groupby('Pclass')['Survived'].sum()
survived_Pclass

Pclass
1    136
2     87
3    119
Name: Survived, dtype: int64

In [74]:
###查看一下不同性别的操作数据
text['Sex'].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [75]:
### 查看下不同舱位人数
text['Pclass'].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

#### 数据小结
- 已统计的人数中，男性是女性人数1.8倍。
- 女性存活人数比男性多124人，女性存活率为74%，男性为19%，危急时刻的人性与绅士风度有所体现。
- 一舱位和三舱位存活人数较多，二舱位较少,实际存活率一舱位63%,二舱位47%，三舱位24%，看来还是靠近上层舱位的人存活率高。

####  利用agg函数和rename函数
`text_fun = text.groupby('Survived').agg({'Sex': 'mean', 'Pclass': 'sum'}).rename(columns=
                            {'Sex': 'mean_sex', 'Pclass': 'count_pclass'})`

In [82]:
###统计在不同等级的票中的不同年龄的船票花费的平均值
text.groupby(['Pclass','Age'])['Fare'].mean().head()

Pclass  Age  
1       0.92     151.5500
        2.00     151.5500
        4.00      81.8583
        11.00    120.0000
        14.00    120.0000
Name: Fare, dtype: float64

In [84]:
###将平均票价和男女存活人数的数据合并，并保存到sex_fare_survived.csv
result = pd.merge(price_mean,survived_sex,on = 'Sex')
result
result.to_csv('sex_fare_survived.csv')

,Fare,Survived
Sex,,
female,44.479818,233
male,25.523893,109


求：不同年龄的总的存活人数，然后找出存活人数的最高的年龄，最后计算存活最高的年龄的存活率（最高年龄存活人数/总人数）

In [88]:
###不同年龄的存活人数
survived_age = text.groupby('Age')['Survived'].sum()
survived_age.head().append(survived_age.tail())

Age
0.42     1
0.67     1
0.75     2
0.83     2
0.92     1
70.00    0
70.50    0
71.00    0
74.00    0
80.00    1
Name: Survived, dtype: int64

In [98]:
###找粗存活人数最大的年龄
max_age_survived = survived_age[survived_age.values == survived_age.max()]
max_age_survived
age = max_age_survived.values.tolist()

Age
24.0    15
Name: Survived, dtype: int64

In [99]:
###存活人数最多的年龄的存活率
survive = text['Survived'].sum()
percent = max_age_survived/survive
percent = percent.values.tolist()
print("最大存活人数的年纪为{}，对应存活率为{}".format(age,percent))

最大存活人数的年纪为[15]，对应存活率为[0.043859649122807015]


### 练习题
1. DataFrame排序的方法有哪些？如果希望数据根据列A为主升序排，列B为辅降序排，应该如何实现？
2. DataFrame和Series可以进行加减乘除运算吗？对应的方法分别是什么？观察一下数据结构不一样的它们是如何实现运算的
3. pandas中很多函数（比如concat都有axis这个参数，它的不同取值分别是什么含义？参数的默认取值是什么？
4. concat，merge，join还有append的用法分别是什么？你能区分它们吗？
5. 对数据groupby后常用的聚合函数有哪些？如何实现对不同的列采取不同的聚合函数？
6. 如何遍历groupby后的数据？
7. groupby默认是将具有相同值的行分成一组，能不能自定义分组函数呢？比如数据有一列城市名，但是我希望根据一线城市、二线城市来分组，如何实现？

### 回答
1. 可以根据索引排序sort_index(),或者值排序sort_values(),by=['A','B']
2. 可以应用apply函数进行运算，也可以直接应用df.add()等函数运算。
3. axis =0/1表示行/列，默认行。
4. concat沿轴，merge属于数据库风格，join沿索引，append沿列方向，注意默认和方向，大部分参数相同。
5. 上述已讲。
6. 直接根据for分组迭代。
7. 可以自定义分组，直接将名字自定义分组例如`city1=np.array(['','',''])`和`city2=np.array(['','',''])`，直接代入`groupby()`内分组，相同的为一组。

### 参考文献

https://blog.csdn.net/brucewong0516/article/details/82707492

https://blog.csdn.net/u012456926/article/details/40353811/

https://blog.csdn.net/weixin_42183408/article/details/89929089

https://blog.csdn.net/qq_38233659/article/details/94666134

https://blog.csdn.net/zzpdbk/article/details/79232661

https://blog.csdn.net/lens___/article/details/83896645

https://blog.csdn.net/weixin_42782150/article/details/90716533

https://zhuanlan.zhihu.com/p/109591980

https://blog.csdn.net/weixin_42946851/article/details/83754417